## Drive Mount

In [ ]:
base_path = "/content/drive/My Drive/yl_tez"

In [ ]:
import os, sys
if 'COLAB_GPU' in os.environ:
    from google.colab import drive
    drive.mount('/content/drive')
else:
    print("Yerelde Çalışıyor.")
    base_path = "."

Mounted at /content/drive


In [ ]:
!pip install peft

In [ ]:
sys.path.append(base_path)
from fonksiyonlar import *
from degiskenler import *
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import DataLoader
from peft import get_peft_model, LoraConfig, TaskType, PeftConfig, PeftModel

## Donanım seç

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# Veri setini oku

In [ ]:
# Veri dosyasını oku
df = pd.read_csv(data_filepath)

In [ ]:
print(df.head().to_markdown())
len(df)

|    | text                                                                                                                                                                                                                                                                                                                                                                                                     |
|---:|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 | Soru: hba1c verisine göre Hastanın HbA1c değeri, diyabet riskini gösteriyor Senior yaş grubundaki birey Hastanın hipertansiyonu tespit edilmemiştir Tetkikler, hastanın kalp rahatsızlığı old

8

# Tokenizer Güncelleme

In [ ]:
texts = df[metin].tolist()
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({"additional_special_tokens": [SORU_TOKEN, CEVAP_TOKEN]})
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/927k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/585k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

# Veri Seti Okuma

In [ ]:
dataset = QADataset(dataframe=df, tokenizer=tokenizer,
                    SORU_TOKEN = SORU_TOKEN, CEVAP_TOKEN = CEVAP_TOKEN, max_len = max_len)

In [ ]:
# Veri setinizin %80'ini eğitim için, %20'sini doğrulama için ayırın
train_size = int(0.8 * len(dataset))
validation_size = len(dataset) - train_size

# Slicing kullanarak veri setini böl
train_dataset = torch.utils.data.Subset(dataset, range(0, train_size))
validation_dataset = torch.utils.data.Subset(dataset, range(train_size, len(dataset)))

# DataLoader'ları oluştur
train_dataloader = DataLoader(train_dataset, batch_size=bs, shuffle=False)
validation_dataloader = DataLoader(validation_dataset, batch_size=bs, shuffle=False)

# ana_veri_seti_deneme
{'input_ids': tensor([[50257,   297,  2267,    17,    67,   451,  1628,   832, 10510,   297,
          2267,    17,    67,  4643, 14160, 12234,  5309,  5466,   293,   589,
         40464,  3115, 10510, 26850,    85,  2537,  1372, 13930, 16884,  9965,
         10510,  4575, 21443,   936,  6482,  2809, 42162,  1178, 14461, 12473,
           509,  4957,  7814,   419,  1063,  6583,  9365,   796, 10510,   263,
          1236,  4643,  3219,   261,  6188,  1486,   458,   221, 50258, 10510,
         14160,   330,   951,   307,  4137,  1684, 13002,   221,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])}

In [ ]:
# DataLoader üzerinden bir iteratör oluştur
data_iterator = iter(train_dataloader)

# İlk yığını al
first_batch = next(data_iterator)

# İlk yığını yazdır
print(first_batch)

{'input_ids': tensor([[50257,   297,  2267,    17,    67,   451,  1628,   832, 10510,   297,
          2267,    17,    67,  4643, 14160, 12234,  5309,  5466,   293,   589,
         40464,  3115, 10510, 26850,    85,  2537,  1372, 13930, 16884,  9965,
         10510,  4575, 21443,   936,  6482,  2809, 42162,  1178, 14461, 12473,
           509,  4957,  7814,   419,  1063,  6583,  9365,   796, 10510,   263,
          1236,  4643,  3219,   261,  6188,  1486,   458,   221, 50258, 10510,
         14160,   330,   951,   307,  4137,  1684, 13002,   221,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

# Model Eğitme

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))
model = get_peft_model(model, peft_config)
model, optimizer, start_epoch, start_batch_index, val_exact_match_list, val_f1_list = load_checkpoint(model, checkpoint_pth,
                                                                                                       checkpoint_name=checkpoint_name, device=device)

Checkpoint şu konumdan yükleniyor ->  /content/drive/My Drive/yl_tez/checkpoint/checkpoint.pth
Checkpoint başarıyla yüklendi: /content/drive/My Drive/yl_tez/checkpoint (Epoch: 11, Batch: 2)


In [ ]:
model = train(model, optimizer, start_epoch, start_batch_index,
              device, train_dataloader, validation_dataloader, 
              val_exact_match_list, val_f1_list, tokenizer, checkpoint_pth,
              epoch_sayisi, checkpoint_name, lr, bs, gorsel_yolu)

Epoch 11 başlıyor...
2 batch atlandı


Processing 11: 100%|██████████| 4/4 [01:06<00:00, 16.50s/it]


 hastada hastanın hastada hastada hastada hastada hastada grubundaki hastada grubundaki hastada grubundaki hastada grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki grubundaki
hastanın diyabeti bulunmamaktadır 
Epoch 11 tamamlandı. F1: 0.06435320973706535, Exact Match: 0.033559120617944145
Checkpoint şuraya kaydedildi -> /

Processing 12:  17%|█▋        | 1/6 [00:29<02:29, 29.95s/it]

In [ ]:
if model is not None:
    # Model ve tokenizer kaydetme
    model.save_pretrained(model_yolu)
    dataset.tokenizer.save_pretrained(model_yolu)

# Model Test Etme

In [ ]:
config = PeftConfig.from_pretrained(model_yolu)
tokenizer = AutoTokenizer.from_pretrained(model_yolu)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
model.resize_token_embeddings(len(tokenizer))
model = PeftModel.from_pretrained(model, model_yolu)
model.to(device)
model.eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
test_et(additional_special_tokens, tokenizer, model, device)

Soru: sa


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Modelin ürettiği cevap:    hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta
Soru: sa
Modelin ürettiği cevap:    hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta hasta h

KeyboardInterrupt: Interrupted by user